In [133]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

# Data Scraping Players Data

My proposed plan for scraping is as follows:
    - Scrape column names for open page ( so there is no redundancy in parsing column names)
    - Develop function to scrape 1 page of player statistics for one season and add to function
    - Use previous function to scrape all pages of player statistics for a whole season
    - Use last three functions to scrape across the past 6 seasons

In [134]:
## Step 1 of our proposed plan
def parse_column_names(link):
    ## requesting the information
    req = requests.get(link)
    soup = BeautifulSoup(req.text)
    ## finding information
    table_tag = soup.find('table')
    col_names = []
    ## looping over column names
    for i in table_tag.find_all('th'):
        col_names.append(i.text)
    return col_names
        

In [135]:
## Step 2
def parse_player_info(link):
    ## requesting the information
    req = requests.get(link)
    soup = BeautifulSoup(req.text)
    ## finding table
    table_tag = soup.find('table')
    ## IF NO MORE PLAYERS BREAK THE LOOP
    if table_tag == None:
        return pd.DataFrame()
    player_infos = []
    player = []
    ## looping over each player info
    for row in table_tag.find_all('tr')[1:]:
        player = []
        for i in row.find_all('td'):
            player.append(i.text)
        player_infos.append(player)
    return pd.DataFrame(player_infos)
        
    

In [136]:
## Step 3
def parse_all_players_per_year(link):
    df = pd.DataFrame()
    ## getting the column names
    col_names = parse_column_names(link % 1)
    ## parsing over pages
    for i in range(1,7):
        ## getting the data and adding to data frame
        parsed_data = parse_player_info(link % i)
        df = pd.concat([df,parsed_data])
    ## changing column names to make it more consistent
    df.columns = col_names
    df = df.rename(columns = {'#': 'SeasonRank'})
    return df
    

In [137]:
## Step 4
def parse_over_years(link):
    ## intiating final data set
    final_df = pd.DataFrame()
    ## looping over years
    for i in ['2014','2015','2016','2017','2018','2019','2020']:
        sublink = link.replace('{}', i)
        df = parse_all_players_per_year(sublink)
        ## making sure to count the years
        df['Year'] = i
        final_df = pd.concat([final_df,df])
    return final_df
        
        

In [138]:
## lets try to load the dataset
players = parse_over_years('https://basketball.realgm.com/nba/stats/{}/Averages/All/points/All/desc/%d/Regular_Season')

Now let's clean the types of the data

In [139]:
players['FG%'] = players['FG%'].astype('float')
players['3PM'] = players['3PM'].astype('float')
players['3PA'] = players['3PA'].astype('float')
players['3P%'] = players['3P%'].astype('float')
players['FTM'] = players['3PM'].astype('float')
players['FTA'] = players['3PA'].astype('float')
players['FT%'] = players['3P%'].astype('float')
players.SeasonRank = players.SeasonRank.astype('int')
players.GP = players.GP.astype('int')
players.MPG = players.MPG.astype('float')
players.FGM = players.FGM.astype('float')
players.FGA = players.FGA.astype('float')
players.TOV = players.TOV.astype('float')
players.PF = players.PF.astype('float')
players.ORB = players.ORB.astype('float')
players.DRB = players.DRB.astype('float')
players.FGA = players.FGA.astype('float')
players.RPG = players.RPG.astype('float')
players.APG = players.APG.astype('float')
players.SPG = players.SPG.astype('float')
players.BPG = players.BPG.astype('float')
players.PPG = players.PPG.astype('float')
players.Year = players.Year.astype('int')



In [140]:
print(players.shape)
players.head()

(3520, 24)


,SeasonRank,Player,Team,GP,MPG,FGM,FGA,FG%,3PM,3PA,...,TOV,PF,ORB,DRB,RPG,APG,SPG,BPG,PPG,Year
0,1,Kevin Durant,OKC,81,38.5,10.5,20.8,0.503,2.4,6.1,...,3.5,2.1,0.7,6.7,7.4,5.5,1.3,0.7,32.0,2014
1,2,Carmelo Anthony,NYK,77,38.7,9.6,21.3,0.452,2.2,5.4,...,2.6,2.9,1.9,6.2,8.1,3.1,1.2,0.7,27.4,2014
2,3,LeBron James,MIA,77,37.7,10.0,17.6,0.567,1.5,4.0,...,3.5,1.6,1.1,5.9,6.9,6.3,1.6,0.3,27.1,2014
3,4,Kevin Love,MIN,77,36.3,8.4,18.5,0.457,2.5,6.6,...,2.5,1.8,2.9,9.6,12.5,4.4,0.8,0.5,26.1,2014
4,5,James Harden,HOU,73,38.0,7.5,16.5,0.456,2.4,6.6,...,3.6,2.4,0.8,3.9,4.7,6.1,1.6,0.4,25.4,2014


# Games Data

Here we load data downloaded from kaggle, but we aren't concerned with all the columns and rows in the data sets, so cleaning has to be done

In [141]:
games = pd.read_csv(os.path.join('Data', 'games.csv'))
game_details = pd.read_csv(os.path.join('Data', 'games_details.csv'))
ranking = pd.read_csv(os.path.join('Data', 'ranking.csv'))
games_players = pd.read_csv(os.path.join('Data', 'players.csv'))
teams = pd.read_csv(os.path.join('Data', 'teams.csv'))

In [142]:
## We are only concerned with games we have data on. Starts on the first game of 2016 NBA Season
final_games = games[pd.to_datetime(games['GAME_DATE_EST']) > '2015-09-20']

## we are not concerned about the actual stats of the game, but rather we want to predict a game before it is
## even played so we get rid of those columns
final_games = final_games[['GAME_DATE_EST','GAME_ID','HOME_TEAM_ID','VISITOR_TEAM_ID','SEASON','HOME_TEAM_WINS']]

Getting the Team Abbreviations so that we can connect our two main datasets

In [143]:
## creating dict to replace values
teams_dict = teams[['TEAM_ID','ABBREVIATION']].set_index('TEAM_ID').to_dict()

## replacing values and renaming columns
final_games[['HOME_TEAM_ABBREV','AWAY_TEAM_ABBREV']] = final_games [['HOME_TEAM_ID','VISITOR_TEAM_ID']].replace(teams_dict['ABBREVIATION'])
final_games = final_games.rename(columns = {'VISITOR_TEAM_ID': 'AWAY_TEAM_ID'})

Getting the previous records of the away and home team (before the game was played)

In [144]:
final_games = (pd.merge(final_games, ranking[['HOME_RECORD','STANDINGSDATE','TEAM_ID']], 
         left_on = ['GAME_DATE_EST','HOME_TEAM_ID'], 
         right_on = ['STANDINGSDATE','TEAM_ID']).drop(['TEAM_ID','STANDINGSDATE'],axis = 1))

final_games = (pd.merge(final_games, ranking[['ROAD_RECORD','STANDINGSDATE','TEAM_ID']], 
         left_on = ['GAME_DATE_EST','AWAY_TEAM_ID'], 
         right_on = ['STANDINGSDATE','TEAM_ID']).drop(['TEAM_ID','STANDINGSDATE'],axis = 1))



Now lets get a list of players who played the game, so that we can use it in predicting the outcome of a game. We
also want to filter out injured/ suspended players.

In [145]:
##filtering out Suspended and Injured
game_details = (game_details[(game_details['COMMENT'].str.contains('Injury', na = False) == False)| 
               (game_details['COMMENT'].str.contains('Suspended', na = False) == False)])
## creating for each game_id, players who played
players_merge = pd.DataFrame(game_details.groupby(['GAME_ID','TEAM_ABBREVIATION'])['PLAYER_NAME'].unique()).reset_index()

## combining two common ids to get home and away team in one column
players_merge = players_merge.groupby('GAME_ID').agg(
    {'TEAM_ABBREVIATION': ', '.join, 'PLAYER_NAME': lambda x: list(x)})

In [146]:
## merging game_details and players for each game
final_games = pd.merge(final_games,players_merge, on = 'GAME_ID')
## home team split helper function
def home_team_split(x):
    if x[6] == (x[10].split(',')[0]):
        index_num = 0
    else:
        index_num = 1
    return pd.Series([x[11][index_num]])
## getting home team players        
final_games['HOME_TEAM_PLAYERS'] = final_games.apply(home_team_split, axis = 1)
## away team split helper function
def away_team_split(x):
    if x[6] == (x[10].split(',')[0]):
        index_num = 1
    else:
        index_num = 0
    return pd.Series([x[11][index_num]])
## getting away team players        
final_games['AWAY_TEAM_PLAYERS'] = final_games.apply(away_team_split, axis = 1)    
## dropping old columns       
final_games = final_games.drop(['TEAM_ABBREVIATION', 'PLAYER_NAME'], axis = 1)    

Lastly, I want to make home and road records an actual integer values so I will turn it into an integer by subtracting wins by lossess and creating an integer record

In [147]:
## making the integers
final_games['HOME_RECORD'] = final_games['HOME_RECORD'].apply(
    lambda x: int(x.split('-')[0]) - int(x.split('-')[1]))
final_games['ROAD_RECORD'] = final_games['ROAD_RECORD'].apply(
    lambda x: int(x.split('-')[0]) - int(x.split('-')[1]))

Now lets organize the date to make it more easier to read and fix datatype

In [148]:
final_games = final_games[['GAME_DATE_EST','GAME_ID','HOME_TEAM_ID','HOME_TEAM_ABBREV','AWAY_TEAM_ID',
                           'AWAY_TEAM_ABBREV','SEASON','HOME_RECORD','ROAD_RECORD','HOME_TEAM_PLAYERS',
                           'AWAY_TEAM_PLAYERS','HOME_TEAM_WINS']]
## converting data type
final_games['GAME_DATE_EST'] = pd.to_datetime(final_games['GAME_DATE_EST'])
final_games['HOME_RECORD'] = final_games['HOME_RECORD'].astype(int)
final_games['ROAD_RECORD'] = final_games['ROAD_RECORD'].astype(int)

In [149]:
final_games

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,HOME_TEAM_ABBREV,AWAY_TEAM_ID,AWAY_TEAM_ABBREV,SEASON,HOME_RECORD,ROAD_RECORD,HOME_TEAM_PLAYERS,AWAY_TEAM_PLAYERS,HOME_TEAM_WINS
0,2020-03-01,21900895,1610612766,CHA,1610612749,MIL,2019,-10,20,"[Miles Bridges, P.J. Washington, Bismack Biyom...","[Wesley Matthews, Giannis Antetokounmpo, Brook...",0
1,2020-03-01,21900896,1610612750,MIN,1610612742,DAL,2019,-15,11,"[Josh Okogie, Juancho Hernangomez, Naz Reid, M...","[Tim Hardaway Jr., Dorian Finney-Smith, Krista...",0
2,2020-03-01,21900897,1610612746,LAC,1610612755,PHI,2019,19,-13,"[Kawhi Leonard, Marcus Morris Sr., Ivica Zubac...","[Glenn Robinson III, Tobias Harris, Al Horford...",1
3,2020-03-01,21900898,1610612743,DEN,1610612761,TOR,2019,17,10,"[Will Barton, Jerami Grant, Nikola Jokic, Gary...","[OG Anunoby, Pascal Siakam, Rondae Hollis-Jeff...",1
4,2020-03-01,21900899,1610612758,SAC,1610612765,DET,2019,-2,-13,"[Harrison Barnes, Nemanja Bjelica, Harry Giles...","[Tony Snell, Christian Wood, John Henson, Svi ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...
6541,2015-10-04,11500005,1610612761,TOR,1610612746,LAC,2015,1,-1,"[DeMarre Carroll, Patrick Patterson, Jonas Val...","[Wesley Johnson, Blake Griffin, DeAndre Jordan...",1
6542,2015-10-04,11500004,1610612747,LAL,1610612762,UTA,2015,-1,1,"[Kobe Bryant, Julius Randle, Roy Hibbert, Jord...","[Gordon Hayward, Derrick Favors, Rudy Gobert, ...",0
6543,2015-10-03,11500002,1610612753,ORL,1610612766,CHA,2015,-1,1,"[Tobias Harris, Channing Frye, Dewayne Dedmon,...","[Michael Kidd-Gilchrist, Marvin Williams, Al J...",0
6544,2015-10-03,11500003,1610612754,IND,1610612740,NOP,2015,-1,1,"[CJ Miles, Paul George, Ian Mahinmi, Monta Ell...","[Dante Cunningham, Anthony Davis, Omer Asik, E...",0


# SAVING THE DATA

Last we save the data as a csv.

In [150]:
final_games.to_pickle('data/final_games.pkl')
players.to_pickle('data/final_players.pkl')